<a href="https://colab.research.google.com/github/aahftab/gemma_rag/blob/main/gemma_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 50.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [ ]:
import keras
import keras_nlp

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")


Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Colab notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Colab notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Colab notebook...


In [ ]:
gemma_lm.summary()

In [ ]:
gemma_lm.generate("What is the meaning of life?", max_length=256)

"What is the meaning of life?\n\nThe question of the meaning of life has perplexed humanity for millennia, with no single, universally accepted answer. However, throughout history, various perspectives and beliefs have emerged, offering insights into the purpose and significance of our existence.\n\n**Philosophical Perspectives:**\n\n* **Existentialism:** Existentialists emphasize the individual's freedom and responsibility in creating their own meaning in a meaningless universe.\n* **Nihilism:** Nihilists believe that life is ultimately pointless and without purpose, and that existence is devoid of meaning.\n* **Stoicism:** Stoics focus on accepting what is beyond our control and finding meaning in what we can control, such as our relationships and personal growth.\n* **Humanism:** Humanists prioritize human values and purpose, seeking meaning in creativity, compassion, and contributing to society.\n\n**Religious Perspectives:**\n\n* **Christianity:** Christians believe that life has 

In [ ]:
gemma_lm.generate("What is llm inference", max_length=512)

'What is llm inference?\n\nLLM inference is a technique used in natural language processing (NLP) to improve the performance of large language models (LLMs) by leveraging auxiliary information from other sources. This information can include human-annotated data, external knowledge bases, or other LLMs.\n\n**How it works:**\n\n1. **Auxiliary information is extracted:** LLMs are trained on a massive dataset and learn representations of the text.\n2. **Auxiliary information is used for inference:** When a new piece of text is presented, the LLM uses the auxiliary information to generate a probability distribution over the possible next words.\n3. **The most likely next words are predicted:** The words with the highest probabilities are selected as the predicted next words.\n\n**Benefits of LLM inference:**\n\n* **Improved accuracy:** LLMs can achieve higher accuracy on tasks such as language translation, text summarization, and question answering by leveraging auxiliary information.\n* *

In [ ]:
!pip install chromadb

In [ ]:
import chromadb
import csv

db = chromadb.Client()

collection = db.get_or_create_collection(name="hadith")

with open("data.csv", 'r') as file:
  csv_reader = csv.DictReader(file)
  data = [row for row in csv_reader]
  id = [row["id"] for row in data]
  hadiths = [row["text"] for row in data]

print(len(hadiths))

34441


In [ ]:
for i in range(0,1000,1000):
  collection.add(
    documents=hadiths[i:i+1000],
    ids=id[i:i+1000]
  )
  print(i)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 56.9MiB/s]


0


In [ ]:
query = "What are the duties of a man in Islam"
results = collection.query(
    query_texts=[query],
    n_results=2
)
result = ''
for i in results['documents'][0]:
  result += i + '\n'

gemma_lm.generate(f"Answer query and context given below \n Query: {query} \n Context: \n {result}", max_length=512)

'Answer query and context given below \n Query: What are the duties of a man in Islam \n Context: \n Narrated Ibn \'Umar:                          Allah\'s Apostle said: Islam is based on (the following) five      (principles):     1. To testify that none has the right to be worshipped but Allah and      Muhammad is Allah\'s Apostle.     2. To offer the (compulsory congregational) prayers dutifully and      perfectly.     3. To pay Zakat (i.e. obligatory charity) .     4. To perform Hajj. (i.e. Pilgrimage to Mecca)     5. To observe fast during the month of Ramadan.\n Narrated Anas bin Malik:                     The Prophet said, "Facilitate things to people (concerning religious matters), and do not make it hard for them and give them good tidings and do not make them run away (from Islam).\n \n\nWhat are the duties of a man in Islam?\n\nAccording to the context, the duties of a man in Islam include testifying that none has the right to be worshipped but Allah and Muhammad is Allah\'s